# **Import Library**

In [ ]:
import pandas as pd
import numpy as np
import random
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from IPython.display import display
import itertools

# **Import Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_cc = pd.read_csv("/content/drive/My Drive/work/kbank/cc.csv", parse_dates=['pos_dt'])
df_demographics = pd.read_csv("/content/drive/My Drive/work/kbank/demographics.csv")
df_kplus = pd.read_csv("/content/drive/My Drive/work/kbank/kplus.csv", parse_dates=['sunday'])
df_test = pd.read_csv("/content/drive/My Drive/work/kbank/test.csv")
df_train = pd.read_csv("/content/drive/My Drive/work/kbank/train.csv")

# **Suyvey Data**

In [ ]:
df_cc.head(2)

,cc_no,pos_dt,cc_txn_amt
0,37069,2018-05-10,5000
1,37069,2018-06-04,12000


In [ ]:
df_demographics.head(2)

,id,cc_no,gender,ocp_cd,age
0,1,1,2,9.0,5
1,1,98397,2,9.0,5


In [ ]:
df_kplus.head(2)

,id,sunday,kp_txn_count,kp_txn_amt
0,14802,2018-01-14,2,2400
1,14802,2018-04-01,9,33900


In [ ]:
display(df_train.head(2))
display(df_train.tail(2))
df_test.head(2)

,id,income
0,1,20000
1,2,106000


,id,income
49998,49999,13000
49999,50000,21000


,id
0,50001
1,50002


In [ ]:
# fill NA ocp_cd
ocp_cd_na_idxs = df_demographics[df_demographics.ocp_cd.isna()].index.tolist()

df_demographics.loc[ocp_cd_na_idxs, 'ocp_cd'] = 2


# **Create Features**

In [ ]:
def get_raw_features():
    demo = df_demographics.drop('cc_no', axis=1)
    demo.drop_duplicates(inplace=True)
    demo = demo.reset_index()
    demo.drop('index', axis=1,inplace=True)
    for index in demo[demo['ocp_cd'].isna()].index:
        demo.iloc[index, 2] = random.choice(range(1, 14))
    return demo

In [ ]:
def get_num_card_feature():
    return df_demographics['id'].value_counts().sort_index().reset_index().rename({'id':  'num_cards'}, axis=1).drop('index', axis=1)

In [ ]:
def get_kplus_monthly_features():
    kplus_copy = df_kplus.copy()
    kplus_copy['month'] = kplus_copy['sunday'].dt.month
    out_df = pd.DataFrame(range(1, 65001), columns=['id'])
    g = kplus_copy.groupby(['id', 'month',]).sum().reset_index()
    month_dict = {
        1: 'jan',
        2: 'feb',
        3: 'mar',
        4: 'apr',
        5: 'may',
        6: 'jun',
    }
    for index, month in month_dict.items():
        tmp = g[g['month'] == index].drop('month', axis=1).rename({'kp_txn_count': f'kp_cnt_{month}', 'kp_txn_amt': f'kp_amt_{month}'}, axis=1)
        out_df = out_df.merge(tmp, how='outer')
        out_df.fillna(0, inplace=True)
    return out_df

In [ ]:
def get_cc_monthly_features():
    cc_no2id = {int(row['cc_no']): int(row['id']) for _, row in df_demographics.iterrows()}
    cc = df_cc.copy()
    cc['month'] = cc['pos_dt'].dt.month
    k = cc.groupby(['cc_no', 'month']).sum().reset_index()
    out_df = pd.DataFrame(range(1, 65001), columns=['id'])
    month_dict = {
            1: 'jan',
            2: 'feb',
            3: 'mar',
            4: 'apr',
            5: 'may',
            6: 'jun',
    }
    for index, month in month_dict.items():
        tmp = k[k['month'] == index].drop('month', axis=1).rename({'cc_txn_amt': f'cc_amt_{month}'}, axis=1)
        tmp['id'] = tmp['cc_no'].map(lambda x: cc_no2id[x])
        tmp.drop(['cc_no'], axis=1, inplace=True)
        tmp = tmp.groupby('id').sum().reset_index()
        out_df = out_df.merge(tmp, how='outer')
    out_df.fillna(0, inplace=True)
    return out_df

In [ ]:
def get_cc_features():
    out_df = pd.DataFrame(range(1, 65001), columns=['id'])
    cc_no2id = {cc: id_ for cc, id_ in  zip(df_demographics['cc_no'], df_demographics['id'])}
    cc = df_cc.copy()
    cc['id'] = cc['cc_no'].map(cc_no2id)
    cc['count'] = 1
    cc.drop('cc_no', axis=1, inplace=True)
    cc_sum = cc.groupby(['id']).sum().reset_index().rename({'cc_txn_amt': 'cc_amt_sum'},axis=1)
    cc_sum['cc_cmt_avg_per_cnt'] = cc_sum['cc_amt_sum'] / cc_sum['count']

    cc_mean = cc.groupby(['id']).mean().reset_index().rename({'cc_txn_amt': 'cc_amt_mean'},axis=1)

    cc_max = cc.groupby(['id']).max().reset_index().rename({'cc_txn_amt': 'cc_amt_max'},axis=1)

    cc_min = cc.groupby(['id']).min().reset_index().rename({'cc_txn_amt': 'cc_amt_min'},axis=1)

    cc_std = cc.groupby(['id']).std().reset_index().rename({'cc_txn_amt': 'cc_amt_std'},axis=1)
    features = [cc_sum, cc_mean, cc_max, cc_min, cc_std]
    for feat in features:
        if 'count' in feat.columns:
            feat.drop(columns='count',inplace=True)
        if 'pos_dt' in feat.columns:
            feat.drop(columns='pos_dt',inplace=True)

        out_df = pd.merge(out_df, feat, on='id', how='outer')
    out_df.fillna(0, inplace=True)
    return out_df

In [ ]:
def get_fwotm_features():
    fwotm = ['2018-01-07', '2018-02-04', '2018-03-04', '2018-04-01', '2018-05-06', '2018-06-03']
    df_kplus['month'] = df_kplus['sunday'].dt.month
    out_df = pd.DataFrame(range(1, 65001), columns=['id'])
    for date in fwotm:
        tmp = df_kplus[df_kplus['sunday'] == date]
        tmp = tmp.groupby('id').sum().drop('month', axis=1).reset_index().rename({'kp_txn_count': f'kp_cnt_{date}', 'kp_txn_amt': f'kplus_amt_{date}'}, axis=1)
        out_df = out_df.merge(tmp, how='outer')
    out_df.fillna(0, inplace=True)
    return out_df

In [ ]:
def get_kp_features():
    out_df = pd.DataFrame(range(1, 65001), columns=['id'])
    kplus = df_kplus.copy()

    kp_sum = kplus.groupby(['id',]).sum().reset_index().rename({'kp_txn_amt': 'kp_amt_sum'}, axis=1)
    kp_sum['kp_amt_avg_per_cnt'] = kp_sum['kp_amt_sum'] / kp_sum['kp_txn_count']

    kp_mean = kplus.groupby(['id',]).mean().reset_index().rename({'kp_txn_amt': 'kp_amt_mean'}, axis=1)

    kp_max = kplus.groupby(['id',]).max().reset_index().rename({'kp_txn_amt': 'kp_amt_max'}, axis=1)

    kp_min = kplus.groupby(['id',]).min().reset_index().rename({'kp_txn_amt': 'kp_amt_min'}, axis=1)

    kp_std = kplus.groupby(['id',]).std().reset_index().rename({'kp_txn_amt': 'kp_amt_std'}, axis=1)

    feats = [kp_sum, kp_mean, kp_max, kp_min, kp_std]
    for feat in feats:

        if 'month' in feat.columns:
            feat.drop('month', axis=1, inplace=True)

        if 'sunday' in feat.columns:
            feat.drop('sunday', axis=1, inplace=True)

        if 'kp_txn_count' in feat.columns:
            feat.drop('kp_txn_count', axis=1, inplace=True)

        out_df = pd.merge(out_df, feat, on='id', how='outer')
    out_df.fillna(0, inplace=True)
    return out_df



In [ ]:
def get_weekend_features():
    cc = df_cc.copy()
    cc_no2id = {cc: id_ for cc, id_ in  zip(df_demographics['cc_no'], df_demographics['id'])}
    cc['id'] = cc['cc_no'].map(cc_no2id)
    cc.drop('cc_no', axis=1,inplace=True)
    out_df = pd.DataFrame(range(1, 65001), columns=['id'])
    cc['is_weekend'] = cc['pos_dt'].dt.dayofweek >=5
    weekend_sum = cc.groupby(['id', 'is_weekend']).sum().reset_index()
    weekend_mean = cc.groupby(['id', 'is_weekend']).mean().reset_index()
    weekend_max = cc.groupby(['id', 'is_weekend']).max().reset_index()
    weekend_std = cc.groupby(['id', 'is_weekend']).std().reset_index()
    weekend_min = cc.groupby(['id', 'is_weekend']).min().reset_index()

    feats = [weekend_sum, weekend_mean, weekend_max, weekend_std, weekend_min]
    names = ['sum', 'mean', 'max', 'std', 'min']
    for feat, name in zip(feats, names):
        feat = (feat[feat['is_weekend']].rename({'cc_txn_amt': f'weekend_{name}'}, axis=1)
                .drop('is_weekend', axis=1).reset_index(drop=True))
        if 'pos_dt' in feat.columns:
            feat.drop('pos_dt', axis=1, inplace=True)
        out_df = pd.merge(out_df, feat, on='id', how='outer')
    out_df.fillna(0, inplace=True)
    return out_df

# **Join Features**

In [ ]:
raw_feats = get_raw_features()
fwotm_feats = get_fwotm_features().drop('id', axis=1)
kp_feats = get_kp_features().drop('id', axis=1)
weekend_feats = get_weekend_features().drop('id', axis=1)
cc_feats = get_cc_features().drop('id', axis=1)

In [ ]:
features = [raw_feats, cc_feats, kp_feats, weekend_feats, fwotm_feats]

for feat in features:
    print(feat.shape)

X = pd.concat(features, axis=1)
# X.drop('id', axis=1,inplace=True)

Xy = X.copy()
# for col in ['gender', 'ocp_cd', 'age']:
#     Xy[col] = Xy[col].astype('category')
# Xy = pd.get_dummies(Xy)
Xy['income'] = df_train['income']



display(Xy.head(2))
Xy.shape

(65000, 4)
(65000, 6)
(65000, 6)
(65000, 5)
(65000, 12)


,id,gender,ocp_cd,age,cc_amt_sum,cc_cmt_avg_per_cnt,cc_amt_mean,cc_amt_max,cc_amt_min,cc_amt_std,kp_amt_sum,kp_amt_avg_per_cnt,kp_amt_mean,kp_amt_max,kp_amt_min,kp_amt_std,weekend_sum,weekend_mean,weekend_max,weekend_std,weekend_min,kp_cnt_2018-01-07,kplus_amt_2018-01-07,kp_cnt_2018-02-04,kplus_amt_2018-02-04,kp_cnt_2018-03-04,kplus_amt_2018-03-04,kp_cnt_2018-04-01,kplus_amt_2018-04-01,kp_cnt_2018-05-06,kplus_amt_2018-05-06,kp_cnt_2018-06-03,kplus_amt_2018-06-03,income
0,1,2,9.0,5,44700.0,8940.000000,8940.000000,10000.0,4700.0,2370.232056,3500.0,1750.000000,1750.0,2000.0,1500.0,353.553391,34700.0,8675.000000,10000.0,2650.000000,4700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1500.0,20000.0
1,2,2,3.0,4,65200.0,1552.380952,1552.380952,14700.0,400.0,2235.112055,210000.0,2019.230769,8400.0,40300.0,100.0,11253.554994,28200.0,1226.086957,4000.0,885.857259,500.0,2.0,600.0,4.0,13700.0,10.0,19500.0,9.0,24200.0,8.0,20600.0,9.0,27700.0,106000.0


(65000, 34)

In [ ]:
X_train = Xy.iloc[:47000, :-1]
y_train = Xy.iloc[:47000, -1].values


X_valid = Xy.iloc[47000:50000, :-1]
y_valid = Xy.iloc[47000:50000, -1].values

X_test = Xy.iloc[50000:, :-1]

# normalize
std_scale = StandardScaler().fit(X_train)
X_train_norm = std_scale.transform(X_train)
X_valid_norm = std_scale.transform(X_valid)
X_test_norm = std_scale.transform(X_test)

print('train:',  X_train.shape, y_train.shape)
print('valid:',  X_valid.shape, y_valid.shape)
print('test:', X_test.shape)

train: (47000, 33) (47000,)
valid: (3000, 33) (3000,)
test: (15000, 33)


In [ ]:
def cal_score(X, y_true, model):
    y_predict = model.predict(X)
    upper = np.square(y_predict - y_true)
    lower = np.square(np.minimum(2 * np.abs(y_true), np.abs(y_predict)) + np.abs(y_true))
    score = 100 - 100 *np.mean(upper/ lower)
    return score

def cal_score_y(y_predict, y_true):
    upper = np.square(y_predict - y_true)
    lower = np.square(np.minimum(2 * np.abs(y_true), np.abs(y_predict)) + np.abs(y_true))
    score = 100 - 100 *np.mean(upper/ lower)
    return score

def print_score(model, norm, features):
    if norm:
        train_score = cal_score(X_train_norm, y_train, model)
        valid_score = cal_score(X_valid_norm, y_valid, model)
    else:
        train_score = cal_score(X_train.loc[:, features], y_train, model)
        valid_score = cal_score(X_valid.loc[:, features], y_valid, model)
    return [train_score, valid_score]

# **XGBoost**

In [ ]:
model = XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.1,
                        max_depth=6, n_estimators=10)

model.fit(X_train, y_train)

cal_score(X_valid, y_valid, model=model)

91.07069544344918

In [ ]:
g = pd.DataFrame({'importance': model.feature_importances_ * 100 , 'feature_name': X_train.columns}).sort_values('importance', ascending=False)
g

,importance,feature_name
4,17.059057,cc_amt_sum
11,15.133004,kp_amt_avg_per_cnt
12,6.854584,kp_amt_mean
3,6.331244,age
13,6.301998,kp_amt_max
16,5.140284,weekend_sum
7,4.425973,cc_amt_max
15,3.479576,kp_amt_std
19,3.052790,weekend_std
9,3.015790,cc_amt_std


# **Predict Data**

In [ ]:
submission = df_test.copy()
submission['income'] = model.predict(X_test)

In [ ]:
submission

,id,income
0,50001,20550.621094
1,50002,19227.238281
2,50003,23266.345703
3,50004,20409.794922
4,50005,26100.302734
...,...,...
14995,64996,17833.652344
14996,64997,19844.226562
14997,64998,19951.867188
14998,64999,26938.722656


# **Export Data**

In [ ]:
# submission.to_csv('/content/drive/My Drive/work/kbank/predict.csv', index=False)